In [1]:
from datetime import datetime
import winsound #for notification sound
from statsmodels.tsa.arima_model import ARIMA

#notification sound parameters
freq = 600
dur = 1000

In [2]:
#financial instrumnet(Rel) specific exchange id

In [3]:
t = "2885"
symbol = "RELIANCE-EQ"

t = "3045"
symbol = "SBIN-EQ"

In [5]:
buy_arr = []
sell_arr = []
profits = []

def buy(p):
    winsound.Beep(freq, dur)
    try:
        orderparams = {
            "variety": "NORMAL",
            "tradingsymbol": "{}".format(symbol),
            "symboltoken": "{}".format(t),
            "transactiontype": "BUY",
            "exchange": "NSE",
            "ordertype": "MARKET",
            "producttype": "INTRADAY",
            "duration": "DAY",
            "squareoff": "0",
            "stoploss": "0",
            "quantity": "1"
            }
        orderId=obj.placeOrder(orderparams)
        print("{}: Buy order id: {} @ price: {}".format(datetime.now().strftime("%H:%M:%S"), orderId, p))
        buy_arr.append(p)
        winsound.Beep(freq, dur)
    except Exception as e:
        print("Order placement failed: {}".format(e.message))
        
def sell(p):
    winsound.Beep(freq, dur)
    try:
        orderparams = {
            "variety": "NORMAL",
            "tradingsymbol": "{}".format(symbol),
            "symboltoken": "{}".format(t),
            "transactiontype": "SELL",
            "exchange": "NSE",
            "ordertype": "MARKET",
            "producttype": "INTRADAY",
            "duration": "DAY",
            "squareoff": "0",
            "stoploss": "0",
            "quantity": "1"
            }
        orderId=obj.placeOrder(orderparams)
        print("{}: Sell order id: {} @ price: {}".format(datetime.now().strftime("%H:%M:%S"), orderId, p))
        sell_arr.append(p)
        profits.append(buy[-1]-p)
        winsound.Beep(freq, dur)
    except Exception as e:
        print("Order placement failed: {}".format(e.message))

In [6]:
data = []

In [7]:
n = 100
p,d,q = (5,1,0)

hold = 0

def data_feed(p):
    data.append(p)
    print(p)
    if len(data) > n:
        model = ARIMA(data, order=(p,d,q))
        model_fit = model.fit(disp=0)
        output = model_fit.forecast()
        yhat = output[0]
        if(yhat > 1.005*data[-1]):
            print("{}: Attempted Buy order @ price: {}".format(datetime.now().strftime("%H:%M:%S"), p))
            #buy_arr.append(p)
            if(hold == 0):
                buy(p)
                hold = 1
                winsound.Beep(freq, dur)
        if(yhat < 0.995*data[-1]):
            print("{}: Attempted Sell order @ price: {}".format(datetime.now().strftime("%H:%M:%S"), p))
            #sell_arr.append(p)
            if(hold > 0):
                sell(p)
                hold = 0
                winsound.Beep(freq, dur)

In [8]:
import time, json
from smartapi import SmartConnect, SmartWebSocket

obj = SmartConnect(api_key = "kTA2vg3s") #trading api: kTA2vg3s; market api: qgQQJ96i 
data = obj.generateSession("S617404","angel996@S")

CLIENT_ID = 'S617404'
FEED_TOKEN = obj.getfeedToken()
token = 'nse_cm|{}'.format(t) #"nse_cm|2885&nse_cm|1594&nse_cm|11536" ; 2885: Rel, 1594: Info, 11536: TCS; 12261: Bhel
task = 'mw' #'mw'|'sfi'|'dp'

ss = SmartWebSocket(FEED_TOKEN, CLIENT_ID)

#y_data = []
winsound.Beep(freq, dur)

def on_ticks(ws, ticks):
	#print("Tick data: {}".format(ticks))
    try:
        price = ticks[0]['ltp']
        data_feed(float(price))
        time.sleep(1)
    except:
        KeyError

def on_connect(ws, response):
	print("on connect")
	ws.send_request(token, task)
	#ws.subscribe([738561, 5633])
	#ws.set_mode(ws.MODE_FULL, [738561])

def on_close(ws, code, reason):
	print("on close")
	ws.stop()


ss.on_ticks = on_ticks
ss.on_connect = on_connect
ss.on_close = on_close

ss.connect()

__on_open################
{'task': 'cn', 'channel': 'NONLM', 'token': '0910060934', 'user': 'S617404', 'acctid': 'S617404'}
2021-06-03 23:40:08.635682 : Start task in the background
{'task': 'hb', 'channel': '', 'token': '0910060934', 'user': 'S617404', 'acctid': 'S617404'}
Error info: 
